# Download dos casos de Raiva 

#### Os casos de Raiva foram baixados a partir do repositório do Sistema de Informação de Agravos de Notificação (SINAN),  que é alimentado, principalmente, pela notificação e investigação de casos de doenças e agravos que constam da lista nacional de doenças de notificação compulsória.  O downlod foi feito a partir do Package Python PySUS, criado para auxiliar no acesso e manipulação de base de dados publicadas pelo DATASUS.

#### Maiores informações em:

 - [SINAN] (https://portalsinan.saude.gov.br/sinan-net)
 - [Raiva Humana] (https://portalsinan.saude.gov.br/raiva-humana)
 - [PySUS] (https://pysus.readthedocs.io/en/latest/)


#### 1 - Importando o método SINAN do pacote PySUS

In [2]:
from pysus.ftp.databases.sinan import SINAN


#### 2 - Carregando os dados a partir do servidor do DataSUS

In [3]:
sinan = SINAN().load()

#### 3 - Lista os arquivos disponíveis para Raiva

In [4]:
arquivos = sinan.get_files(dis_code=["RAIV"])

#### 4 - Download dos arquivos disponíveis

In [5]:
import pandas as pd
import pyarrow
import pyarrow.parquet as pq

In [6]:
import os

In [7]:
parquets = sinan.download(arquivos)

100%|██████████| 14.3k/14.3k [00:00<00:00, 6.05MB/s]


In [8]:
files = []
input_folder = "/home/limafb/pysus/"
target_path = "/home/limafb/pysus/sinan_raiva.parquet"

for file_name in os.listdir(input_folder):
    if file_name.startswith('RAIV'):
        files.append(pq.read_table(os.path.join(input_folder, file_name)))

with pq.ParquetWriter(
    target_path, 
    files[0].schema, 
    version='2.6', 
    compression='gzip', 
    use_dictionary=True, 
    data_page_size=2097152, #2MB 
    write_statistics=True) as writer:
        for f in files:
              writer.write_table(f)

In [9]:
sinan_raiva = pq.read_table("sinan_raiva.parquet")

In [10]:
df = sinan_raiva.to_pandas()

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1679 entries, 0 to 1678
Data columns (total 94 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   TP_NOT      1679 non-null   object
 1   ID_AGRAVO   1679 non-null   object
 2   DT_NOTIFIC  1679 non-null   object
 3   SEM_NOT     1679 non-null   object
 4   NU_ANO      1679 non-null   object
 5   SG_UF_NOT   1679 non-null   object
 6   ID_MUNICIP  1679 non-null   object
 7   ID_REGIONA  1679 non-null   object
 8   ID_UNIDADE  1679 non-null   object
 9   DT_SIN_PRI  1679 non-null   object
 10  SEM_PRI     1679 non-null   object
 11  ANO_NASC    1679 non-null   object
 12  NU_IDADE_N  1679 non-null   object
 13  CS_SEXO     1679 non-null   object
 14  CS_GESTANT  1679 non-null   object
 15  CS_RACA     1679 non-null   object
 16  CS_ESCOL_N  1679 non-null   object
 17  SG_UF       1679 non-null   object
 18  ID_MN_RESI  1679 non-null   object
 19  ID_RG_RESI  1679 non-null   object
 20  ID_PAIS 

#### 05 - Exportando em CSV para criar um indicador no Visão por Município e por Ano

In [ ]:
# Reordenando as colunas e agrupandoos dados por ano e município para criar um indicador no Visão
sinan_raiva_mun_ano_2007_2021 = merged.groupby(['code_muni', 'NU_ANO']).size().reset_index(name='valor')

In [ ]:
# Converte a coluna 'code_muni' de float para int em uma nova coluna 'geocode'
sinan_raiva_mun_ano_2007_2021['geocode'] = sinan_raiva_mun_ano_2007_2021['code_muni'].astype('Int64')

In [ ]:
#Criar o campo 'data' e padronizar a partir do valor da coluna 'NU_ANO'
sinan_raiva_mun_ano_2007_2021['data'] = "31/12/" + sinan_raiva_mun_ano_2007_2021['NU_ANO'].astype(str)

In [ ]:
sinan_raiva_mun_ano_2007_2021_visao = sinan_raiva_mun_ano_2007_2021[['geocode', 'valor', 'data']]

In [ ]:
# Remover os registros com valores na
sinan_raiva_2007_2021_visao = sinan_raiva_2007_2021_visao.dropna()